In [ ]:
import pandas as pd
import requests
import RiotAnalyzer as Ra
from datetime import datetime

In [ ]:
api_key_dhil = 'RGAPI-9c72c116-9d41-4245-8af9-e431b87bf6cc'
game_version = "13.17.1"

In [ ]:
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9,id;q=0.8,ja;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": f"{api_key_dhil}"
}
url_template = "https://{region_code}.api.riotgames.com/lol/{endpoint}?{query}"

In [142]:
def get_puuid(name:str, region_code:str="NA1"):
    url = url_template.format(region_code=region_code, endpoint="summoner/v4/summoners/by-name/"+name, query="page=1")
    response = requests.get(url, headers=header)
    if response.status_code != 200:
        print("Error: ", response.status_code)
    else:
        name_data = response.json()
    puuid = name_data["puuid"]
    return puuid

    

In [143]:
def get_mastery(puuid:str, region_code:str="NA1"):
    url = url_template.format(region_code=region_code, endpoint=f"champion-mastery/v4/champion-masteries/by-puuid/{puuid}", query="page=1")
    response = requests.get(url, headers=header)
    if response.status_code != 200:
        print("Error: ", response.status_code)
    else:
        return response.json()

In [ ]:
ra = Ra.RiotAnalyzer(api_key_dhil, version=game_version)

In [ ]:
champ_dict = ra.get_champion_dict()

In [144]:
def search_mastery_by_summoner_name(name, region_code:str="NA1"):
    puuid = get_puuid(name, region_code)
    mastery = get_mastery(puuid,region_code)
    df = pd.DataFrame(mastery)
    df["championName"] = df["championId"].apply(lambda x: champ_dict[str(x)])
    df = df[["championName", "championLevel", "championPoints", "lastPlayTime"]]
    df["lastPlayDate"] = df["lastPlayTime"].apply(lambda x: datetime.fromtimestamp(x/1000))
    df["lastPlayTime"] = df["lastPlayTime"].apply(lambda x: datetime.fromtimestamp(x/1000).strftime("%b %d %H:%M"))
    df["percent"] = df["championPoints"]/df["championPoints"].sum()
    df["percent"] = df["percent"].apply(lambda x: "{:.2%}".format(x))
    df.index += 1
    df.index.name = "Rank"
    return df

In [146]:
df = search_mastery_by_summoner_name("GawrGura OP", "SG").sort_values(by="lastPlayDate",ascending=False)
df.head(15)

ConnectionError: HTTPSConnectionPool(host='sg.api.riotgames.com', port=443): Max retries exceeded with url: /lol/summoner/v4/summoners/by-name/GawrGuraOP?page=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B3F91A7F10>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [126]:
df[df["championLevel"] == 7]

,championName,championLevel,championPoints,lastPlayTime,percent
Rank,,,,,
1,Master Yi,7,506471,Jun 04 09:06,20.56%
2,Lux,7,350410,Jun 04 04:59,14.23%
3,Nunu & Willump,7,261787,Jun 04 09:39,10.63%


In [ ]:
this only works for NA rn, gotta change it so we can adjust that as a parameter

In [ ]:
yea, we can do that later tho
u said there was a dictionary of champname and roles? 
alr imma look for it


In [85]:
def get_champion_dict(version="13.17.1"):
    """Gets a dictionary of champion names and their IDs

    Args:
        version (str): The version of the game to get the champion data for

    Returns:
        dict: A dictionary of champion names and their IDs
    """
    url = f"http://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion.json"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Error {response.status_code} when querying champion data")
    data = response.json()
    # Take data and put inside a pandas dataframe
    df = pd.DataFrame(data["data"]).T
    return df

In [139]:
cd = get_champion_dict()
cd.get('key')['Aatrox']

'266'

In [87]:
cd.loc["Neeko"]

version                                              13.17.1
id                                                     Neeko
key                                                      518
name                                                   Neeko
title                                  the Curious Chameleon
blurb      Hailing from a long lost tribe of vastaya, Nee...
info       {'attack': 1, 'defense': 1, 'magic': 9, 'diffi...
image      {'full': 'Neeko.png', 'sprite': 'champion2.png...
tags                                         [Mage, Support]
partype                                                 Mana
stats      {'hp': 610, 'hpperlevel': 104, 'mp': 450, 'mpp...
Name: Neeko, dtype: object

In [113]:
cd.loc["Vex"]['tags']

['Mage']

In [115]:
cd['tags'] that has general class but class doesn't always = lane
at the very least we need to include junglers
problem is like a lot of champs can play a lot of roles
wait I got an idea 
https://www.leagueofgraphs.com/champions/stats/shyvana
we can include all the roles with above like 10-15% usage rate

SyntaxError: unterminated string literal (detected at line 1) (48993790.py, line 1)

In [121]:
url = "https://www.leagueofgraphs.com/champions/stats/shyvana"
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Error {response.status_code} when querying champion data")
data = response.json()

Exception: Error 403 when querying champion data